<a href="https://colab.research.google.com/github/taguch1s/study/blob/main/pactical-nlp-ja/03_text_embedding_to_06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

@see https://github.com/oreilly-japan/practical-nlp-ja/tree/master/ch03

# 1.テキストのOne-Hotエンコーディング

このノートブックでは、one-hotエンコーディングを実装します。実際のプロジェクトでは、scikit-learnの実装を使うことのほうが多いでしょう。

In [1]:
documents = [
    "Dog bites man.",
    "Man bites dog.",
    "Dog eats meat.",
    "Man eats food."
]
processed_docs = [doc.lower().replace(".", "") for doc in documents]
processed_docs

['dog bites man', 'man bites dog', 'dog eats meat', 'man eats food']

In [2]:
# ボキャブラリの構築
vocab = {}
count = 0
for doc in processed_docs:
    for word in doc.split():
        if word not in vocab:
            count = count + 1
            vocab[word] = count
print(vocab)

{'dog': 1, 'bites': 2, 'man': 3, 'eats': 4, 'meat': 5, 'food': 6}


In [3]:
# 与えたテキストに対応するone-hot表現を取得
# 単語がボキャブラリに存在する場合、その表現が返される。
# 存在しない場合は、ゼロのリストが返される。
def get_onehot_vector(somestring):
    onehot_encoded = []
    for word in somestring.split():
        temp = [0] * len(vocab)
        if word in vocab:
            temp[vocab[word] - 1] = 1 # リストのインデックスは1ではなく0から始まるため-1している
        onehot_encoded.append(temp)
    return onehot_encoded

コーパスのテキストに対して、one-hot表現を取得してみましょう。

In [5]:
print(processed_docs[1])
get_onehot_vector(processed_docs[1])

man bites dog


[[0, 0, 1, 0, 0, 0], [0, 1, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0]]

同じボキャブラリを使って、ランダムなテキストに対するone-hot表現を取得してみましょう。

In [6]:
get_onehot_vector("man and dog are good")

[[0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0]]

In [7]:
get_onehot_vector("man and man are good")

[[0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0]]

## scikit-learnを用いたOne-hotエンコーディング

ここでは、scikit-learnのOneHotEncoderを用いて、one-hotエンコーディングをしてみましょう。

具体的には、次のことを試します。

* One-Hotエンコーディング: one-hotエンコーディングでは、ボキャブラリの各単語wに、1から|V|の間の一意の整数IDとしてwidを与える。各単語は、0と1からなるV次元の二値ベクトルで表現される。

* ラベルエンコーディング: ラベルエンコーディングでは、コーパスの各単語wを0からn-1までの数値に変換する。ここでnはコーパスに含まれる一意な単語数を意味する。

参考

- [OneHotEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html)
- [LabelEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html) respectively.








In [8]:
S1 = 'dog bites man'
S2 = 'man bites dog'
S3 = 'dog eats meat'
S4 = 'man eats food'

In [9]:
from itertools import chain
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

data = [S1.split(), S2.split(), S3.split(), S4.split()]
values = list(chain.from_iterable(data))
print(f"The data: {values}")

# Label Encoding
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
print(f"Label Encoded: {integer_encoded}")

# One-Hot Encoding
onehot_encoder = OneHotEncoder()
onehot_encoded = onehot_encoder.fit_transform(data).toarray()
print(f"Onehot Encoded Matrix:\n{onehot_encoded}")

The data: ['dog', 'bites', 'man', 'man', 'bites', 'dog', 'dog', 'eats', 'meat', 'man', 'eats', 'food']
Label Encoded: [1 0 4 4 0 1 1 2 5 4 2 3]
Onehot Encoded Matrix:
[[1. 0. 1. 0. 0. 0. 1. 0.]
 [0. 1. 1. 0. 1. 0. 0. 0.]
 [1. 0. 0. 1. 0. 0. 0. 1.]
 [0. 1. 0. 1. 0. 1. 0. 0.]]


# 2.Bag of Words

In [10]:
documents = [
    "Dog bites man.",
    "Man bites dog.",
    "Dog eats meat.",
    "Man eats food."
]
processed_docs = [doc.lower().replace(".", "") for doc in documents]
processed_docs

['dog bites man', 'man bites dog', 'dog eats meat', 'man eats food']

scikit-learnのCountVectorizerを使うと、bag of wordsを簡単に作成できる

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

# 文書のリストを確認
print("Our corpus: ", processed_docs)

count_vect = CountVectorizer()
# BoW表現の構築
bow_rep = count_vect.fit_transform(processed_docs)

# ボキャブラリの確認
print("Our vocabulary: ", count_vect.vocabulary_)

# 最初の2つの文書に対するBoWを確認
print("BoW representation for 'dog bites man': ", bow_rep[0].toarray())
print("BoW representation for 'man bites dog': ", bow_rep[1].toarray())

# 新しいテキストに対するBoWを取得
temp = count_vect.transform(["dog and dog are friends"])
print("BoW representation for 'dog and dog are friends': ", temp.toarray())

Our corpus:  ['dog bites man', 'man bites dog', 'dog eats meat', 'man eats food']
Our vocabulary:  {'dog': 1, 'bites': 0, 'man': 4, 'eats': 2, 'meat': 5, 'food': 3}
BoW representation for 'dog bites man':  [[1 1 0 0 1 0]]
BoW representation for 'man bites dog':  [[1 1 0 0 1 0]]
BoW representation for 'dog and dog are friends':  [[0 2 0 0 0 0]]


文章中の登場回数を考慮しない場合は以下のように `CountVectorizer`を`binary=True`で初期化する

In [12]:
count_vect = CountVectorizer(binary=True)
bow_rep_bin = count_vect.fit_transform(processed_docs)
temp = count_vect.transform(["dog and dog are friends"])
print("Bow representation for 'dog and dog are friends':", temp.toarray())

Bow representation for 'dog and dog are friends': [[0 1 0 0 0 0]]


# 3.Bag of N-grams

In [14]:
# init
documents = [
    "Dog bites man.",
    "Man bites dog.",
    "Dog eats meat.",
    "Man eats food."
]
processed_docs = [doc.lower().replace(".", "") for doc in documents]
processed_docs

['dog bites man', 'man bites dog', 'dog eats meat', 'man eats food']

In [15]:
# CountVectorizerを用いてnグラム（n=1, 2, 3）のベクトル化
count_vect = CountVectorizer(ngram_range=(1, 3))

# BoN表現の構築
bow_rep = count_vect.fit_transform(processed_docs)

# 語彙の確認
print("Our vocabulary: ", count_vect.vocabulary_)

# 新しいテキストに対するBoNの確認
temp = count_vect.transform(["dog and dog are friends"])
print("Bow representation for 'dog and dog are friends':", temp.toarray())


Our vocabulary:  {'dog': 3, 'bites': 0, 'man': 12, 'dog bites': 4, 'bites man': 2, 'dog bites man': 5, 'man bites': 13, 'bites dog': 1, 'man bites dog': 14, 'eats': 8, 'meat': 17, 'dog eats': 6, 'eats meat': 10, 'dog eats meat': 7, 'food': 11, 'man eats': 15, 'eats food': 9, 'man eats food': 16}
Bow representation for 'dog and dog are friends': [[0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


# 4.TF-IDF
テキスト内の単語に重要性という概念を初めて導入した手法

In [18]:
documents = [
    "Dog bites man.",
    "Man bites dog.",
    "Dog eats meat.",
    "Man eats food."
]
processed_docs = [doc.lower().replace(".", "") for doc in documents]
processed_docs

['dog bites man', 'man bites dog', 'dog eats meat', 'man eats food']

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
bow_rep_tfidf = tfidf.fit_transform(processed_docs)

# ボキャブラリの全単語に対するIDF
print("IDF for all words in the vocabulary",tfidf.idf_)
print("-" * 10)

# ボキャブラリの全単語
print("All words in the vocabulary",tfidf.get_feature_names_out())
print("-"*10)

# 全文書に対するTFIDF
print("TFIDF representation for all documents in our corpus\n",bow_rep_tfidf.toarray())
print("-"*10)

temp = tfidf.transform(["dog and man are friends"])
print("Tfidf representation for 'dog and man are friends':\n", temp.toarray())

IDF for all words in the vocabulary [1.51082562 1.22314355 1.51082562 1.91629073 1.22314355 1.91629073]
----------
All words in the vocabulary ['bites' 'dog' 'eats' 'food' 'man' 'meat']
----------
TFIDF representation for all documents in our corpus
 [[0.65782931 0.53256952 0.         0.         0.53256952 0.        ]
 [0.65782931 0.53256952 0.         0.         0.53256952 0.        ]
 [0.         0.44809973 0.55349232 0.         0.         0.70203482]
 [0.         0.         0.55349232 0.70203482 0.44809973 0.        ]]
----------
Tfidf representation for 'dog and man are friends':
 [[0.         0.70710678 0.         0.         0.70710678 0.        ]]


scikit-learn >= 1.0.x から`get_feature_names()` は `get_feature_names_out()`に変更

https://stackoverflow.com/questions/70215049/attributeerror-tfidfvectorizer-object-has-no-attribute-get-feature-names-out

# 5.学習済みモデルを用いた分散表現

In [1]:
!pip install -q gensim==4.1.2 spacy==3.1.2

In [2]:
import os
import psutil # 実行中のプロセスやシステムのリソース活用について調べるため
import time
import warnings # 生成される様々な警告を無視するため

warnings.filterwarnings("ignore")
process = psutil.Process(os.getpid())
mem = psutil.virtual_memory()

## 事前学習済み埋め込みのダウンロード

In [ ]:
!wget -P /tmp/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

In [ ]:
from gensim.models import KeyedVectors
path = '/tmp/input/GoogleNews-vectors-negative300.bin.gz'

pre = process.memory_info().rss
print(f"モデル読み込み前のメモリ使用量（GB）: {pre/(10**9):.2f}")
print('-' * 10)

start_time = time.time() # タイマーのスタート
ttl = mem.total          # 利用可能なメモリ量

#　モデルの読み込み
vectors = KeyedVectors.load_word2vec_format(path, binary=True)
print(f"読み込みにかかった時間（秒）: {time.time() - start_time:.2f}")
print('-' * 10)

print('モデルの読み込み完了')
print('-' * 10)

post = process.memory_info().rss
print(f"モデル読み込み後のメモリ使用量（GB）: {post/(10**9):.2f}")
print('-' * 10)

print(f"メモリ使用量の増加率: {post/pre*100:.2f}%")
print('-' * 10)

print(f"ボキャブラリ数: {len(vectors.key_to_index)}")

In [ ]:
# 類似した単語の検索
vectors.most_similar('beautiful')

In [ ]:
vectors.most_similar('toronto')

In [ ]:
# ベクトル表現の確認
vectors['beautiful']

# 6.gensimを用いた単語埋め込みの学習

In [3]:
import time
import warnings

from gensim.models import FastText, Word2Vec, KeyedVectors
from gensim.corpora.wikicorpus import WikiCorpus
warnings.filterwarnings('ignore')

In [4]:
# gensimのword2vecでは、リストのリスト形式での学習データが必要です。
# 各リストが1つの文書を表し、文書はトークンのリストで表されます。
corpus = [
    ['dog', 'bites', 'man'],
    ['man', 'bites', 'dog'],
    ['dog', 'eats', 'meat'],
    ['man', 'eats', 'food']
]

## Continuous Bag of Words (CBOW)

CBOWの主なタスクは、文脈語を与えたときに、その中心語を正しく予測できる言語モデルを構築することです。 gensimのWord2Vecクラスに学習データとハイパーパラメータを指定することで学習できます。 学習アルゴリズムは、sgパラメータで指定します。CBOWを使うなら0、Skip-gramを使うなら1を指定します。その他のパラメータについては、以下のドキュメントを参照してください。

[gensim.models.word2vec.Word2Vec](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec)

In [11]:
model_cbow.wv.index_to_key

['man', 'dog', 'eats', 'bites', 'food', 'meat']

In [12]:
# モデルの学習
model_cbow = Word2Vec(corpus, min_count=1, sg=0)

# モデルのサマリー
print(model_cbow)

# ボキャブラリの表示
words = list(model_cbow.wv.index_to_key)
print(words)

# 単語ベクトルの取得
print(model_cbow.wv['dog'])

Word2Vec(vocab=6, vector_size=100, alpha=0.025)
['man', 'dog', 'eats', 'bites', 'food', 'meat']
[-8.6196875e-03  3.6657380e-03  5.1898835e-03  5.7419385e-03
  7.4669183e-03 -6.1676754e-03  1.1056137e-03  6.0472824e-03
 -2.8400505e-03 -6.1735227e-03 -4.1022300e-04 -8.3689485e-03
 -5.6000124e-03  7.1045388e-03  3.3525396e-03  7.2256695e-03
  6.8002474e-03  7.5307419e-03 -3.7891543e-03 -5.6180597e-04
  2.3483764e-03 -4.5190323e-03  8.3887316e-03 -9.8581640e-03
  6.7646410e-03  2.9144168e-03 -4.9328315e-03  4.3981876e-03
 -1.7395747e-03  6.7113843e-03  9.9648498e-03 -4.3624435e-03
 -5.9933780e-04 -5.6956373e-03  3.8508223e-03  2.7866268e-03
  6.8910765e-03  6.1010956e-03  9.5384968e-03  9.2734173e-03
  7.8980681e-03 -6.9895042e-03 -9.1558648e-03 -3.5575271e-04
 -3.0998408e-03  7.8943167e-03  5.9385742e-03 -1.5456629e-03
  1.5109634e-03  1.7900408e-03  7.8175711e-03 -9.5101865e-03
 -2.0553112e-04  3.4691966e-03 -9.3897223e-04  8.3817719e-03
  9.0107834e-03  6.5365066e-03 -7.1162102e-04  7.7

In [13]:
# 類似度の計算
print("Similarity between eats and bites:", model_cbow.wv.similarity('eats', 'bites'))
print("Similarity between eats and man:", model_cbow.wv.similarity('eats', 'man'))

Similarity between eats and bites: -0.0134970825
Similarity between eats and man: -0.05235437


In [14]:
model_cbow.wv.most_similar('meat')

[('food', 0.13887985050678253),
 ('bites', 0.13149003684520721),
 ('eats', 0.06422408670186996),
 ('dog', 0.009391166269779205),
 ('man', -0.05987630784511566)]

In [ ]:
# モデルの保存
model_cbow.save('model_cbow.bin')

# モデルの読み込み
new_model_cbow = Word2Vec.load('model_cbow.bin')
print(new_model_cbow)

CBOWの方が学習時間はかかるが，精度で勝るということらしい

## Wikiコーパスを用いた埋め込みの学習

ここまでで、小さなデータを使ってモデルを学習する方法は紹介しました。 次は、より大きなデータを使ってモデルを学習してみましょう。 ただ、そのすべてを利用すると全体で数GBを超えるので、今回はその一部だけを使って、word2vecとfastTextの埋め込みを学習します。コーパスは以下のページからダウンロードできます。

https://dumps.wikimedia.org/jawiki/

## データのダウンロード


In [ ]:
!mkdir -p data/ja/
!wget -P data/ja/ https://dumps.wikimedia.org/jawiki/20240501/jawiki-20240501-pages-articles6.xml-p4307948p4926292.bz2

## パッケージのインストール

日本語の場合、学習前に単語分割が必要なので、形態素解析器「MeCab」のラッパーである「fugashi」をインストールします。

In [ ]:
!pip install fugashi[unidic-lite]

## 前処理

インストールを終えたら、単語分割用の関数を定義します。 学習データの準備にgensimのWikiCorpusを利用するため、その内部で必要とされる形式の関数を定義しています。パラメータは以下の通りです。

* content (str): Wikiのマークアップを除去した文字列
* token_min_len (int): 最小のトークン長
* token_max_len (int): 最大のトークン長
* lower (bool): 文字列を小文字化するか否か

In [ ]:
from fugashi import Tagger
from gensim.corpora.wikicorpus import TOKEN_MAX_LEN, WikiCorpus
from gensim import utils
fugger = Tagger('-Owakati')
TOKEN_MIN_LEN = 1

def tokenize(
    content,
    token_min_len=TOKEN_MIN_LEN,
    token_max_len=TOKEN_MAX_LEN,
    lower=True
  ):
    return [
        utils.to_unicode(token.surface) for token in fugger(content.lower() if lower else content)
    ]

...